In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil
import healpy

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [2]:
def assign_lya_qso(initial_mtl_file, pixweight_file):
    targets = Table.read(initial_mtl_file)

    pixweight, header = fits.getdata(pixweight_file, 'PIXWEIGHTS', header=True)
    hpxnside = header['HPXNSIDE']

    theta_w, phi_w = healpy.pix2ang(hpxnside, pixweight['HPXPIXEL'], nest=True)


    hpxnside_sample = 64 # pixel area on which we will sample the lyaqso
    npix_sample = healpy.nside2npix(hpxnside_sample)
    pixnumber_sample = healpy.ang2pix(hpxnside_sample, theta_w, phi_w, nest=True)

    subpixels = {} # store the pixels at the resolution in the input target catalog that are included within the pixels at the new resolution.
    for i in range(npix_sample):
        ii_sample = pixnumber_sample==i
        subpixels[i] =  healpy.ang2pix(hpxnside, theta_w[ii_sample], phi_w[ii_sample], nest=True)
    
    
    # redefine the covered area for the new pixels from the higher resolution FRACAREA
    covered_area = np.ones(npix_sample)
    for i in range(npix_sample):
        sum_weight = np.sum(pixweight['FRACAREA'][subpixels[i]])
        if sum_weight>0.0:
            covered_area[i] = np.sum(pixweight['FRACAREA'][subpixels[i]]**2)/np.sum(pixweight['FRACAREA'][subpixels[i]])
        else:
            covered_area[i] = 0.0

    theta_s, phi_s = healpy.pix2ang(hpxnside_sample, np.arange(npix_sample), nest=True)

    pixelarea_sample = healpy.pixelfunc.nside2pixarea(hpxnside_sample, degrees=True)
    n_lya_qso_in_pixel = np.int_(covered_area * 50 * pixelarea_sample)

    # compute angular coordinates from the targets
    targets_phi = np.deg2rad(targets['RA'])
    targets_theta = np.deg2rad(90.0-targets['DEC'])

    # find the pixnumber to which the target belongs (in the hpxnside_sample resolution)
    pixnumber_targets = healpy.ang2pix(hpxnside_sample, targets_theta, targets_phi, nest=True)

    
    # what target are QSOs?
    is_qso = (targets['DESI_TARGET'] & desi_mask.QSO)!=0

    # list of unique pixels covered by the targets
    pixnumber_target_list = list(set(pixnumber_targets)) # list of pixelsIDs covered by the targets in the new resolution

    n_qso_per_pixel_targets = np.zeros(len(pixnumber_target_list))
    for i in range(len(pixnumber_target_list)):
        ii_targets = is_qso & (pixnumber_targets==pixnumber_target_list[i])
        n_qso_per_pixel_targets[i] = np.count_nonzero(ii_targets)
    
    n_lya_desired_pixel_targets = np.random.poisson(n_lya_qso_in_pixel[pixnumber_target_list])

    # Generate the boolean array to determine whether a target is a lyaqso or not
    is_lya_qso = np.repeat(False, len(targets))
    n_targets = len(targets)
    target_ids = np.arange(n_targets)

    for i in range(len(pixnumber_target_list)):
        ii_targets = is_qso & (pixnumber_targets==pixnumber_target_list[i])
        n_qso_in_pixel = np.count_nonzero(ii_targets)
        n_lya_desired = np.random.poisson(n_qso_in_pixel)
        if n_lya_desired >= n_qso_in_pixel:
            is_lya_qso[ii_targets] = True
        else:
            #print(len(target_ids[ii_targets]), n_lya_desired)
            ii_lya_qso = np.random.choice(target_ids[ii_targets], n_lya_desired, replace=False)
            is_lya_qso[ii_lya_qso] = True
    return is_lya_qso

In [4]:
initial_Ztl_file = "targets/subset_dr8_mtl_dark_gray_NGC.fits"
pixweight_file = "/project/projectdirs/desi/target/catalogs/dr8/0.31.1/pixweight/pixweight-dr8-0.31.1.fits"
is_lya_qso = assign_lya_qso(initial_mtl_file, pixweight_file)

In [5]:
initial_truth_file = "targets/subset_truth_dr8_mtl_dark_gray_NGC.fits"
if not os.path.exists(initial_truth_file):
    import desitarget.mock.mockmaker as mb
    from desitarget.targetmask import desi_mask, bgs_mask, mws_mask

    targets = Table.read(initial_mtl_file)
    colnames = list(targets.dtype.names)
    print(colnames)
    nobj = len(targets)
    truth = mb.empty_truth_table(nobj=nobj)[0]
    print(truth.keys())

    for k in colnames:
        if k in truth.keys():
            print(k)
            truth[k][:] = targets[k][:]

    nothing = '          '
    truth['TEMPLATESUBTYPE'] = np.repeat(nothing, nobj)

    masks = ['MWS_ANY', 'BGS_ANY', 'STD_FAINT', 'STD_BRIGHT','ELG', 'LRG', 'QSO', ]
    dict_truespectype = {'BGS_ANY':'GALAXY', 'ELG':'GALAXY', 'LRG':'GALAXY', 'QSO':'QSO', 
                    'MWS_ANY':'STAR', 'STD_FAINT':'STAR', 'STD_BRIGHT':'STAR'}
    dict_truetemplatetype = {'BGS_ANY':'BGS', 'ELG':'ELG', 'LRG':'LRG', 'QSO':'QSO', 
                        'MWS_ANY':'STAR', 'STD_FAINT':'STAR', 'STD_BRIGHT':'STAR'}
    dict_truez = {'BGS_ANY':0.2, 'ELG':1.5, 'LRG':0.7, 'QSO':2.0, 
                        'MWS_ANY':0.0, 'STD_FAINT':0.0, 'STD_BRIGHT':0.0}

    for m in masks:
        istype = (targets['DESI_TARGET'] & desi_mask.mask(m))!=0
        print(m, np.count_nonzero(istype))
        truth['TRUESPECTYPE'][istype] = np.repeat(dict_truespectype[m], np.count_nonzero(istype))
        truth['TEMPLATETYPE'][istype] = np.repeat(dict_truetemplatetype[m], np.count_nonzero(istype))
        truth['MOCKID'][istype] = targets['TARGETID'][istype]
        truth['TRUEZ'][istype] = dict_truez[m]

        
    truth['TRUEZ'][is_lya_qso] = 3.0
    # Check that all targets have been assigned to a class
    iii = truth['MOCKID']==0
    assert np.count_nonzero(iii)==0
    
    print('writing truth')
    truth.write(initial_truth_file, overwrite=True)
    print('done truth')

In [43]:
truth = Table.read(initial_truth_file)

In [44]:
ii = (truth['TRUEZ']>2.0)

In [45]:
set(truth['TEMPLATETYPE'][ii])

{'QSO'}